In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../data/Club_Performance.csv')

In [ ]:
df

In [ ]:
df.columns

In [5]:
def calc_l1_points(row):
    l1_count = min(row['Level 1s'], 4)
    return l1_count * 100

def calc_l2_points(row):
    l2_total = row['Level 2s'] + row['Add. Level 2s']
    l2_count = min(l2_total, 2)
    return l2_count * 200

def calc_l3_points(row):
    l3_count = min(row['Level 3s'], 2)
    return l3_count * 300

def calc_l4_points(row):
    return 400 if row['Level 4s, Path Completions, or DTM Awards'] >= 1 else 0

def calc_l5_points(row):
    return 500 if row['Add. Level 4s, Path Completions, or DTM award'] >= 1 else 0

def calc_cot_r1_points(row):
    return 200 if row['Off. Trained Round 1'] >= 7 else 0

def calc_cot_r2_points(row):
    return 200 if row['Off. Trained Round 2'] >= 7 else 0



In [6]:
def assign_club_group(row):
    members = row['Active Members']
    if members < 12:
        return 'Group 1'
    elif 12 <= members <= 20:
        return 'Group 2'
    elif 21 <= members <= 40:
        return 'Group 3'
    elif 41 <= members <= 100:
        return 'Group 4'
    else:
        return 'Unknown'


In [7]:
# Apply group label
df['Group'] = df.apply(assign_club_group, axis=1)

# Optional: Add Name and Description as new columns
group_meta = {
    'Group 1': {
        'Name': 'Spark Clubs',
        'Description': 'Small but full of potential, these clubs are just igniting.'
    },
    'Group 2': {
        'Name': 'Rising Stars',
        'Description': 'Gaining traction, these clubs are building energy and cohesion.'
    },
    'Group 3': {
        'Name': 'Powerhouse Clubs',
        'Description': 'Well-established, these clubs thrive on teamwork and synergy.'
    },
    'Group 4': {
        'Name': 'Pinnacle Clubs',
        'Description': 'Large, vibrant clubs at the peak of influence and activity.'
    },
    'Unknown': {
        'Name': 'Undefined',
        'Description': 'Club size not in defined range.'
    }
}

df['Group Name'] = df['Group'].map(lambda g: group_meta[g]['Name'])
df['Group Description'] = df['Group'].map(lambda g: group_meta[g]['Description'])


In [ ]:
df['L1 Points'] = df.apply(calc_l1_points, axis=1)
df['L2 Points'] = df.apply(calc_l2_points, axis=1)
df['L3 Points'] = df.apply(calc_l3_points, axis=1)
df['L4 Points'] = df.apply(calc_l4_points, axis=1)
df['L5 Points'] = df.apply(calc_l5_points, axis=1)
df['COT R1 Points'] = df.apply(calc_cot_r1_points, axis=1)
df['COT R2 Points'] = df.apply(calc_cot_r2_points, axis=1)
df['Total COT Points'] = df['COT R1 Points'] + df['COT R2 Points']

df['Total Club Points'] = df[['L1 Points', 'L2 Points', 'L3 Points', 'L4 Points', 'L5 Points', 'Total COT Points']].sum(axis=1)
df

In [ ]:
# Sort by Group and Total Club Points descending
df_sorted = df.sort_values(['Group', 'Total Club Points'], ascending=[True, False]).copy()

# Assign rank within each group
df_sorted['Group Rank'] = df_sorted.groupby('Group')['Total Club Points'].rank(
    method='dense', ascending=False
).astype(int)


top_5_by_group = df_sorted[df_sorted['Group Rank'] <= 5][[
    'Group', 'Group Name', 'Club Name', 'Active Members', 'Total Club Points', 'Group Rank'
]].reset_index(drop=True)

top_5_by_group